In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import os

# Setup
output_dir = "dms_sim"
os.makedirs(output_dir, exist_ok=True)

# Seed for reproducibility
np.random.seed(42)

# Generate x-axis (time)
x = np.arange(1, 100)

# Step 1: Generate log curve
true_signal = np.log(x)

# Step 2: Normalize log curve to [0, 1]
true_signal_normalized = (true_signal - true_signal.min()) / (true_signal.max() - true_signal.min())

# Prepare list to collect image filenames organized by noise level
image_grid = []

# Define noise levels and alpha values
noise_levels = [0.01, 0.03, 0.05, 0.1, 0.2]
alpha_values = np.linspace(0.01, 1.0, 10)

# Dynamic Savitzky-Golay window parameters
scaling_factor = 2
min_window = 5
max_window = 21

for noise_level in noise_levels:
    image_row = []

    # Step 3: Add controlled noise
    noise = np.random.normal(loc=0, scale=noise_level, size=len(x))
    noisy_signal = true_signal_normalized + noise

    # Step 4: Clip to [0, 1]
    noisy_signal = np.clip(noisy_signal, 0, 1)

    # Prepare DataFrame
    df = pd.DataFrame({"x": x, "CDSS": noisy_signal})

    for alpha in alpha_values:
        # Dynamically compute Savitzky-Golay window length
        window_length = int(np.clip(1 / alpha * scaling_factor, min_window, max_window))
        if window_length % 2 == 0:
            window_length += 1  # ensure odd

        # Apply Savitzky-Golay filter (1st derivative)
        try:
            savgol_derivative = savgol_filter(df["CDSS"], window_length=window_length, polyorder=2, deriv=1, delta=1.0)
        except ValueError:
            # Fallback if not enough points
            savgol_derivative = np.zeros_like(df["CDSS"])

        # Compute EWMA methods
        ewma_then_diff = df["CDSS"].ewm(alpha=alpha, adjust=True).mean().diff().fillna(0)
        diff_then_ewma = df["CDSS"].diff().fillna(0).ewm(alpha=alpha, adjust=True).mean()

        # Create dual-axis plot
        fig, ax1 = plt.subplots(figsize=(6, 4))

        # Primary axis: noisy signal
        ax1.plot(df["x"], df["CDSS"], label="Noisy CDSS (log curve)", color="black", linewidth=1.5, alpha=0.6)
        ax1.set_xlabel("Time")
        ax1.set_ylabel("CDSS Score (Original)", color="black")
        ax1.tick_params(axis='y', labelcolor="black")

        # Secondary axis: derivatives
        ax2 = ax1.twinx()
        ax2.plot(df["x"], ewma_then_diff, label="EWMA → diff", linestyle="--", color="blue", linewidth=2)
        ax2.plot(df["x"], diff_then_ewma, label="diff → EWMA", linestyle=":", color="orange", linewidth=2)
        ax2.plot(df["x"], savgol_derivative, label=f"Savitzky-Golay (window {window_length})", linestyle="-.", color="green", linewidth=2)

        ax2.set_ylabel("Change (Delta)", color="gray")
        ax2.tick_params(axis='y', labelcolor="gray")

        # Combined legend
        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels + labels2, loc="upper right", fontsize=7)

        plt.title(f"Noise = {noise_level:.2f} | Alpha = {alpha:.2f}")
        plt.grid(True)
        plt.tight_layout()

        # Save figure
        filename = f"noise_{noise_level:.2f}_alpha_{alpha:.2f}.png"
        full_path = os.path.join(output_dir, filename)
        plt.savefig(full_path, dpi=150)
        plt.close()

        image_row.append(filename)

    image_grid.append(image_row)

# Generate HTML table
html_content = """
<html>
<head>
<style>
body {
    margin: 0;
    padding: 0;
}
table {
    table-layout: fixed;
    border-collapse: collapse;
    width: auto;
    white-space: nowrap;
}
td {
    text-align: center;
    vertical-align: top;
    padding: 0;
    border: none;
}
img {
    width: 250px;
    height: auto;
    display: block;
    margin: auto;
    border: 1px solid #ddd;
}
th {
    padding: 4px;
    background-color: #f9f9f9;
    font-family: Arial, sans-serif;
    font-size: 12px;
}
</style>
</head>
<body>
<table>
<tr>
    <th>Noise \\ Alpha</th>
"""

# Add alpha header
for alpha in alpha_values:
    html_content += f"<th>α = {alpha:.2f}</th>"
html_content += "</tr>\n"

# Add rows of images per noise level
for noise_level, image_row in zip(noise_levels, image_grid):
    html_content += f"<tr><th>Noise = {noise_level:.2f}</th>"
    for image_file in image_row:
        html_content += f'<td><img src="{image_file}"><br><small>{image_file}</small></td>'
    html_content += "</tr>\n"

html_content += """
</table>
</body>
</html>
"""

# Save HTML file
html_file_path = os.path.join(output_dir, "summary.html")
with open(html_file_path, "w") as f:
    f.write(html_content)

print(f"✅ All plots saved to {output_dir}/")
print(f"✅ HTML summary saved to {html_file_path}")